# Фреймворк PyTorch для разработки искусственных нейронных сетей
## Урок 1. Основные понятия и структуры PyTorch

Выполнить действия [урока по tesorflow](https://www.tensorflow.org/tutorials/quickstart/beginner) c помощью PyTorch

In [ ]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train.shape, x_test.shape

11501568/11490434 [==============================] - 0s 0us/step


((60000, 28, 28), (10000, 28, 28))

In [ ]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader

In [ ]:
num_classes = 10
input_size = 784
batch_size = 64
lr = 0.0001
epochs = 3

In [ ]:
T = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
 
X_train = torchvision.datasets.MNIST(root='/datasets', train=True, download=True, transform=T)
train_loader = DataLoader(dataset=X_train, batch_size=batch_size, shuffle=True)
 
X_test = torchvision.datasets.MNIST(root='/datasets', train=False, download=True, transform=T)
test_loader = DataLoader(dataset=X_test, batch_size=batch_size, shuffle=True)

In [ ]:
class tf_to_pt(nn.Module):

      def __init__(self, n_inputs):

        # вызов конструктора родительского класса
        super().__init__()

        # определяю tf.keras.layers.Dense(128, activation='relu')
        self.hid1 = nn.Linear(n_inputs, 128) 
        nn.init.kaiming_uniform_(self.hid1.weight, nonlinearity='relu')
        self.act1 = nn.ReLU()

        # определяю tf.keras.layers.Dense(10)
        self.hid2 = nn.Linear(128, 10)


      def forward(self,X):

        # определяю tf.keras.layers.Flatten(input_shape=(28, 28))
        X = torch.flatten(X)

        #для слоя 1
        X = self.hid1(X)
        X = self.act1(X)

        # определяю tf.keras.layers.Dropout(0.2)
        X = nn.functional.dropout(X, p=0.2, training=True)
        
        #для слоя 2
        X = self.hid2(X)

        return X

In [ ]:
# исполняю predictions = model(x_train[:1]).numpy()
# predictions

model = tf_to_pt(28*28)
model.double()
predictions = model.forward(torch.tensor(x_train[:1]))
predictions.detach().numpy()

array([-0.24605149,  0.12983101,  0.20331848,  0.35044482,  0.14750785,
        0.05325338,  0.2026781 ,  0.30464382, -0.03064095,  0.30739203])

In [ ]:
# исполняю tf.nn.softmax(predictions).numpy()
nn.functional.softmax(predictions)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


tensor([0.0669, 0.0974, 0.1048, 0.1214, 0.0991, 0.0902, 0.1048, 0.1160, 0.0830,
        0.1163], dtype=torch.float64, grad_fn=<SoftmaxBackward0>)

In [67]:
class not_mine_neural_network(nn.Module):
    def __init__(self, input_size, num_classes):
        super(not_mine_neural_network, self).__init__()
        self.fc1 = nn.Linear(in_features=input_size, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=num_classes)
       
    def forward(self, x):
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = nn.functional.dropout(x, p=0.2, training=True)
        x = self.fc2(x)
        return x

In [68]:
# исполняю loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# исполняю loss_fn(y_train[:1], predictions).numpy() - эту строчку не знаю как исполнить отдельно
# исполняю model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
# исполняю model.fit(x_train, y_train, epochs=5)
model = not_mine_neural_network(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for epoch in range(epochs):
    for batch, (data, target) in enumerate(train_loader):
       
        # Reshaping to suit our model
        data = data.reshape(data.shape[0], -1)
       
        # Forward propagation
        score = model(data)
        loss = criterion(score, target)
       
        # Backward propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [70]:
# исполняю model.evaluate(x_test,  y_test, verbose=2)
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
 
    with torch.no_grad():
        for x, y in loader:
            x = x.reshape(x.shape[0], -1)
 
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
           
        if num_samples == 60000:
            print(f"Train accuracy = "
                  f"{float(num_correct) / float(num_samples) * 100:.2f}")
        else:
            print(f"Test accuracy = "
                  f"{float(num_correct) / float(num_samples) * 100:.2f}")
 
    model.train()
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Train accuracy = 91.11
Test accuracy = 91.58


In [90]:
# исполняю probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
# исполняю probability_model(x_test[:5])
scores = []
for data, _ in train_loader:
  if len(scores) >= 5:
    break
  data = data.reshape(data.shape[0], -1)
  score = model(data)
  scores.append(score)
nn.Softmax(scores)

Softmax(
  dim=[tensor([[ -4.4745,  -5.3496,  -0.0522,  -1.8855,  -5.1595,  -3.9088,  -8.4809,
             9.3721,  -2.3822,   2.8423],
          [  1.3862,  -8.4670,   0.9159,  -6.8085,  -0.2331,   0.5675,  10.4183,
            -8.7676,  -1.6317,  -4.9968],
          [ -6.1584,   4.8187,   1.3169,  -0.6734,  -3.1099,  -4.6797,  -0.9196,
            -0.1136,   0.6876,  -2.5185],
          [  5.5421,  -6.9313,  -1.5500,  -0.9652,  -4.3819,   1.6022,  -0.4044,
            -2.4509,   0.8721,  -2.1333],
          [  7.8857, -10.4621,  -1.5141,  -0.7914,  -5.7121,   5.2059,  -5.5424,
            -4.2140,   2.2409,  -1.2810],
          [ -2.4146,  -3.3434,  -2.1908,  -2.6312,  -3.0693,  -0.1070,  -2.7123,
            -4.7114,   4.6114,   0.1453],
          [ -0.7805,  -3.3904,  -2.9495,  -3.5021,  -1.9511,   7.0079,  -3.2939,
            -2.5002,   0.1361,  -4.2383],
          [ -7.6517,   6.8476,  -0.0502,  -0.3218,  -5.2615,  -1.8189,  -1.6945,
            -1.7683,   0.2701,  -2.2344],
  